## Experiment with finding the bit start times, using traces previously collected by otbn_traces.ipynb:

In [ ]:
import numpy as np
#waves = np.load('waves_p256_streamed.npy')
waves = np.load('waves_p256_streamed_half1half0.npy')
#waves = np.load('waves_p256_streamed_patterned.npy')
#waves = np.load('waves_p256_streamed_32bitblocks.npy')
#waves = [np.load('waves_p256_100M.npy')]
#wave = [np.load('waves_p256_100M_2s.npy')]

In [ ]:
#samples = 100000
samples = len(waves[0])
base = 0

In [ ]:
import holoviews as hv
from holoviews.operation import decimate
from holoviews.operation.datashader import datashade
hv.extension('bokeh')
#datashade(hv.Curve(waves[0][base:base+samples]-waves[9][base:base+samples])).opts(width=2000, height=900)
datashade(hv.Curve(waves[0][base:base+samples])).opts(width=2000, height=900)

### HPF helps clean up the trace, especially at higher samples rates:

In [ ]:
import numpy as np
import pandas as pd
from scipy import signal

def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq 
    b, a = signal.butter(order, normal_cutoff, btype='high', analog=False)
    return b, a

def butter_highpass_filter(data, cutoff, fs, order=9):
    b, a = butter_highpass(cutoff, fs, order=order)
    y = signal.filtfilt(b, a, data)
    return y

filtered_wave = butter_highpass_filter(waves[0], 1e6, 100e6) # for streamed 25M capture
#filtered_wave = butter_highpass_filter(waves[0], 6e6, 100e6) # for NON-streamed 100M capture

In [ ]:
#samples = len(waves[0])
samples = 220000
base = 0

import holoviews as hv
from holoviews.operation import decimate
from holoviews.operation.datashader import datashade, shade, dynspread
hv.extension('bokeh')

w0 = datashade(hv.Curve(waves[0][base:base+samples]), cmap=['green'])
wf = datashade(hv.Curve(filtered_wave[base:base+samples]), cmap=['black'])

#(w0 * wf).opts(width=2000, height=900)
(wf).opts(width=2000, height=600)

In [ ]:
def contiguous_regions(condition):
    """Finds contiguous True regions of the boolean array "condition". Returns
    a 2D array where the first column is the start index of the region and the
    second column is the end index."""

    # Find the indicies of changes in "condition"
    d = np.diff(condition.astype(int))
    idx, = d.nonzero() 

    # We need to start things after the change in "condition". Therefore, 
    # we'll shift the index by 1 to the right.
    idx += 1

    if condition[0]:
        # If the start of condition is True prepend a 0
        idx = np.r_[0, idx]

    if condition[-1]:
        # If the end of condition is True, append the length of the array
        idx = np.r_[idx, condition.size] # Edit

    # Reshape the result into two columns
    idx.shape = (-1,2)
    return idx


# Method 1:
Use idle periods as a marker.
Works really well for 25M, less so for 100M.

In [ ]:
#for 25M streamed:
THRESHOLD = 0.1
MIN_RUN_LENGTH = 500

# for 100M NOT streamed:
#THRESHOLD = 0.015
#MIN_RUN_LENGTH = 60

STOP=len(filtered_wave)
#STOP=360000
condition = np.abs(filtered_wave[:STOP]) < THRESHOLD

# Print the start and stop indices of each region where the absolute 
# values of x are below 1, and the min and max of each of these regions
results = contiguous_regions(condition)
#print(len(results))
goods = results[np.where(results[:,1] - results[:,0] > MIN_RUN_LENGTH)]
print(len(goods))

In [ ]:
# for debug:
last_stop = 0
for g in goods:
    start = g[0]
    stop = g[1]
    l = stop-start
    delta = start - last_stop
    if 13000 < delta < 18000:
        stat = 'ok'
    else:
        stat = 'OOOOPS?!?'
    print('%8d %8d %8d %8d %s' % (l, delta, start, stop, stat))
    last_stop = stop

### Use these runs to guess at bit start times (25M edition):

In [ ]:
bit_starts = []
idle_count = 0
max_active = 10
for i, idle_period in enumerate(goods[1:]):
    #print('Processing start=%10d... ' % goods[i][0], end='')
    if goods[i][0] - goods[i-1][1] <= max_active:
        idle_count += 1
        #print('idle_count=%d for start=%d, stop=%d' % (idle_count, goods[i][0], goods[i][1]))
        if idle_count == 4:
            #print('Found bit starting at %d' % goods[i+1][0])
            bit_starts.append(goods[i][1]+1)
            idle_count = 0
    else:
        #print('resetting idle_count for start=%d, stop=%d because delta=%d (%d, %d)' % (goods[i][0], goods[i][1], goods[i][0] - goods[i-1][1], goods[i][0], goods[i-1][1]))
        idle_count = 1

### Use these runs to guess at bit start times (100M edition):

In [ ]:
raw_starts = []
for i in range(3, len(goods), 2):
    raw_starts.append(goods[i][1])

### For 100M only: try to make the bit start times more accurate by using the single isolated large peak that's about 650 samples in:
hmm, not sure if this actually helps...

In [ ]:
wstart = 500
wend = 700

wstart = 1550
wend = 1620

base = np.argmax(filtered_wave[raw_starts[0]+wstart:raw_starts[0]+wend])
bit_starts = [raw_starts[0]]
for s in raw_starts[1:]:
    loc = np.argmax(filtered_wave[s+wstart:s+wend])
    offset = base-loc
    print(offset)
    bit_starts.append(s + offset)

# Sanity check the results:

In [ ]:
if len(bit_starts) != 256:
    print('Oops, found %d bits :-/' % len(bit_starts))
    
duration = bit_starts[1] - bit_starts[0]
print("First bit duration: %d cycles" % duration)
for b in range(2, len(bit_starts)):
    d = bit_starts[b] - bit_starts[b-1] 
    if d != duration:
        print("Unexpected duration for bit %d: %d cycles" % (b-1, d))


# Superimpose all the bits!
Plot overlayed bit traces to visualize alignment and guess at success of time extraction:

In [ ]:
bits = []
bit_size = bit_starts[1] - bit_starts[0]
for start in bit_starts:
    bits.append(filtered_wave[start:start+bit_size])

In [ ]:
len(bits)

In [ ]:
# Can't plot more than 22 bits due to palette:
#numbits = len(bits)
numbits = 22

import holoviews as hv
from holoviews.operation import decimate
from holoviews.operation.datashader import datashade, shade, dynspread
from holoviews import opts
import datashader as ds

hv.extension('bokeh')

lines = {i: hv.Curve(np.asarray(bits[i])) for i in range(len(bits[:numbits]))}
linespread = dynspread(datashade(hv.NdOverlay(lines, kdims='k'), aggregator=ds.by('k', ds.count())))
linespread.opts(opts.RGB(width=2400, height=1200))


In [ ]:
# Can plot all the bits, but it's slow:
#numbits = len(bits)
numbits = 20

import holoviews as hv
from holoviews.operation import decimate
from holoviews.operation.datashader import datashade, shade, dynspread
hv.extension('bokeh')

xrange = range(duration)

from operator import mul
from functools import reduce

curves = [hv.Curve(zip(xrange, filtered_wave[bit_starts[i]:bit_starts[i]+duration])) for i in range(numbits)]

reduce(mul, curves).opts(width=2000, height=900)

# Average 'one' and 'zero':
Let's see if simple difference-of-means reveals anything.

In [ ]:
duration

In [ ]:
avg_trace = np.zeros(duration)
avg_ones = np.zeros(duration)
avg_zeros = np.zeros(duration)

for i, start in enumerate(bit_starts):
    avg_trace += waves[0][start:start+duration]
    if i < 128:
        avg_ones += waves[0][start:start+duration]
    else:
        avg_zeros += waves[0][start:start+duration]

avg_trace /= len(bit_starts)
avg_ones /= len(bit_starts)/2
avg_zeros /= len(bit_starts)/2


In [ ]:
import holoviews as hv
from holoviews.operation import decimate
from holoviews.operation.datashader import datashade, shade, dynspread
hv.extension('bokeh')

xrange = range(duration)

cavg_all = datashade(hv.Curve(avg_trace), cmap=['black'])
cavg_ones = datashade(hv.Curve(avg_ones), cmap=['blue'])
cavg_zeros = datashade(hv.Curve(avg_zeros), cmap=['green'])

cdiff = datashade(hv.Curve((avg_ones - avg_zeros)*10), cmap=['red'])

#(cavg_all * cavg_ones * cavg_zeros).opts(width=2000, height=900)
(cdiff).opts(width=2000, height=600)

In [ ]:
markers = np.where((avg_ones - avg_zeros) > 0.005)[0]

In [ ]:
markers

In [ ]:
# save markers for attacking other traces:
np.save('markers_25M.npy', markers)

In [ ]:
scores = []
for b in bit_starts:
    score = 0
    for marker in markers:
        score += filtered_wave[b + marker]
    scores.append(score)

In [ ]:
cscores = hv.Curve(scores)
(cscores).opts(width=2000, height=600)